## Overture Maps Data Download Test
### Gavin Rolls

NOTE: Use urbsim kernel to avoid parse error

In [1]:
#Library Imports - using DuckDB for Overture Import
import duckdb
import pandas as pd

In [3]:
#Config SQL
%pip install ipython-sql duckdb duckdb-engine jupysql --quiet
%pip install --upgrade grpcio --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [10]:
%%sql      
INSTALL httpfs;

LOAD httpfs;

INSTALL spatial;

LOAD spatial;

SET s3_region='us-west-2';

,Success


### Demo Direct Loading of Overture Data (Demo from Data Extraction Docs - Overture Maps)

In [ ]:
#Takes forever - just a proof of concept. It does work, though

# %%sql

# CREATE OR REPLACE VIEW admins_view AS (
#     SELECT
#         *
#     FROM
#         read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=admins/type=*/*', filename=true, hive_partitioning=1)
# );

# COPY (
#     SELECT
#            admins.id,
#            admins.subtype,
#            admins.iso_country_code_alpha_2,
#            admins.admin_level,
#            areas.area_id,
#            names.primary AS primary_name,
#            sources[1].dataset AS primary_source,
#            ST_GeomFromWkb(areas.area_geometry) AS geometry
#       FROM admins_view AS admins
#       INNER JOIN (
#            SELECT
#                   id AS area_id,
#                   locality_id,
#                   geometry AS area_geometry
#            FROM admins_view
#        ) AS areas ON areas.locality_id == admins.id
#       WHERE admins.admin_level = 1
#  ) TO 'countries_overture.geojson'
# WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

### Demo Direct Loading of POIs

In [13]:
%%sql

COPY(
    SELECT
       id,
       names.primary as primary_name,
       bbox.xmin as x,
       bbox.ymin as y,
       ST_GeomFromWKB(geometry) as geometry,
       categories.main as main_category,
       sources[1].dataset AS primary_source,
       confidence
    FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/type=*/*', filename=true, hive_partitioning=1)
    WHERE main_category = 'mountain' AND confidence > .90
    ORDER BY confidence DESC
) TO 'overture_places_mountains_gt90.shp'
WITH (FORMAT GDAL, DRIVER 'GeoJSON');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
